In [ ]:
!pip -q install pandas numpy

import os
import re
import pandas as pd


SPS_PATH = "/content/2023-SADC-SPSS-Input-Program.sps"
DAT_PATH = "/content/sadc_2023_national.dat"

if not os.path.exists(SPS_PATH):
    raise FileNotFoundError(f"Missing: {SPS_PATH}. Upload the .sps to Colab.")
if not os.path.exists(DAT_PATH):
    raise FileNotFoundError(f"Missing: {DAT_PATH}. Upload the .dat to Colab.")

print("Using SPS:", SPS_PATH)
print("Using DAT:", DAT_PATH)


def parse_sps_fixed_width(sps_path: str):
    txt = open(sps_path, "r", encoding="utf-8", errors="ignore").read()


    pattern = re.compile(r"\n\s*([A-Za-z0-9_]+)\s+(\d+)\s*-\s*(\d+)", re.MULTILINE)
    matches = pattern.findall(txt)
    if not matches:
        raise ValueError("Could not parse fixed-width specs from the SPSS file.")

    seen = set()
    names, colspecs = [], []
    for var, a, b in matches:
        if var in seen:
            continue
        seen.add(var)
        start = int(a) - 1
        end = int(b)
        names.append(var.lower())
        colspecs.append((start, end))

    if len(names) < 50:
        raise ValueError(f"Parsed only {len(names)} columns. Check SPS/DAT pair.")

    return names, colspecs

names, colspecs = parse_sps_fixed_width(SPS_PATH)
print("Parsed columns:", len(names))
print("First 12 columns:", names[:12])


df = pd.read_fwf(DAT_PATH, colspecs=colspecs, names=names, dtype=str)
print("Loaded shape:", df.shape)


def resolve_var(preferred: str, fallbacks: list[str]) -> str:
    """Return the first variable name found in df.columns from preferred/fallbacks."""
    preferred = preferred.lower()
    if preferred in df.columns:
        return preferred
    for fb in fallbacks:
        fb = fb.lower()
        if fb in df.columns:
            return fb
    return None


selected_vars = {
    "grade": "grade",
    "sleep_8plus": "qn85",           # sleep (8+ hours) per your earlier mapping
    "sad_hopeless": "qn26",          # sad/hopeless :contentReference[oaicite:5]{index=5}

    "miss_school_unsafe": "qn14",    # unsafe-related missed school :contentReference[oaicite:6]{index=6}
    "bullied_school": "qn24",        # bullied at school :contentReference[oaicite:7]{index=7}
    "electronic_bullied": "qn25",    # electronically bullied :contentReference[oaicite:8]{index=8}


    "pe_daily": "qndlype",
    "soda_2plus": "qnsoda2",
    "breakfast_7day": "qnbk7day",


    "age": "age",
    "sex": "sex",
    "race7": "race7"
}

missing = [v for v in selected_vars.values() if v not in df.columns]
if missing:
    raise KeyError(
        f"These columns are missing in df: {missing}\n"
        f"Try printing candidates: [c for c in df.columns if c.startswith('qn')]"
    )

out = df[list(selected_vars.values())].copy()
out.columns = list(selected_vars.keys())


out = out.replace(r"^\s*$", pd.NA, regex=True).replace(".", pd.NA)

csv_name = "yrbs_2023_sadc_vars.csv"
out.to_csv(csv_name, index=False)

print("✅ Saved:", csv_name, " shape:", out.shape)
out.head()
